In [1]:
import pymssql
import pandas as pd
import numpy as np
import pyodbc
import warnings 
warnings.filterwarnings("ignore")
from tqdm import tqdm
import os
from numpy import abs
from numpy import log
from numpy import sign
from scipy.stats import rankdata
import datetime as dt

#導入數據
import math
from vnpy_datamanager.engine import ManagerEngine
from vnpy.trader.engine import BaseEngine, MainEngine, EventEngine
from vnpy.trader.constant import (Exchange, Interval)

#回測模塊
from datetime import datetime
import matplotlib.pyplot as plt
import binance

from vnpy_portfoliostrategy.backtesting import BacktestingEngine
from vnpy_portfoliostrategy.strategies.mason_strategy import MasonStrategy

In [12]:
#下載歷史資料
client = binance.Client()      #先將API實例化方便取用

#撰寫幣安爬蟲API
def data_download(symbol, symbol_class, interval, start_str) :

    if symbol_class == 'spot' :
        klines = client.get_historical_klines(symbol, interval, start_str)
    if symbol_class == 'future' :
        klines = client.futures_historical_klines(symbol, interval, start_str)
    data = pd.DataFrame(klines)
    data.columns = ['open_time', 'open', 'high', 'low', 'close',
    'volume', 'close_time', 'qav', 'num_trades',
    'taker_base_vol', 'taker_base_vol', 'ignore']    
    data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.close_time]
    data.index = data.index + dt.timedelta(seconds = 0.001)
    data.reset_index(inplace=True)
    data.set_index('index',inplace=True)
    data = data[['open','high','low','close','volume']]
    data['open'] = data['open'].astype(float)
    data['high'] = data['high'].astype(float)
    data['low'] = data['low'].astype(float)
    data['close'] = data['close'].astype(float)
    data['volume'] = data['volume'].astype(float)
    data["open_interest"] = 0
    return data    

In [13]:
#下載歷史資料 data1=期貨 , data2=現貨
data1 = data_download('BTCUSDT','future','1m','2022-11-01')
data2 = data_download('BTCUSDT','spot','1m','2022-11-01')

In [23]:
data1['open'] = data1['open'].astype(float)
data1['high'] = data1['high'].astype(float)
data1['low'] = data1['low'].astype(float)
data1['close'] = data1['close'].astype(float)
data1['volume'] = data1['volume'].astype(float)
data1['turnover'] = data1['volume'] * data1['close']
data1["open_interest"] = 0
data1.index.name = None
data1.index = data1.index.strftime('%Y-%m-%d %H:%M:%S')
data1 = data1.reset_index(drop = False)
data1 = data1.rename(columns={'index':'datetime'})

In [25]:
data1.to_csv(r'C:\Users\chiehlinchen\Documents\GitHub\Big_project\data\BTCUSDT_FUTURE.csv')

In [26]:
data2['open'] = data2['open'].astype(float)
data2['high'] = data2['high'].astype(float)
data2['low'] = data2['low'].astype(float)
data2['close'] = data2['close'].astype(float)
data2['volume'] = data2['volume'].astype(float)
data2['turnover'] = data2['volume'] * data2['close']
data2["open_interest"] = 0
data2.index.name = None
data2.index = data2.index.strftime('%Y-%m-%d %H:%M:%S')
data2 = data2.reset_index(drop = False)
data2 = data2.rename(columns={'index':'datetime'})

In [28]:
data2.to_csv(r'C:\Users\chiehlinchen\Documents\GitHub\Big_project\data\BTCUSDT_SPOT.csv')

In [29]:
manager = ManagerEngine(MainEngine,EventEngine)

找不到数据服务驱动vnpy_，使用默认的RQData数据服务


In [31]:
manager.import_data_from_csv(r'C:\Users\chiehlinchen\Documents\GitHub\Big_project\data\BTCUSDT_FUTURE.csv','BTCUSDT_FUTURE',Exchange.LOCAL,Interval.MINUTE,"Asia/Shanghai","datetime","open","high","low","close","volume","turnover","open_interest","%Y-%m-%d %H:%M:%S")
manager.import_data_from_csv(r'C:\Users\chiehlinchen\Documents\GitHub\Big_project\data\BTCUSDT_SPOT.csv','BTCUSDT_SPOT',Exchange.LOCAL,Interval.MINUTE,"Asia/Shanghai","datetime","open","high","low","close","volume","turnover","open_interest","%Y-%m-%d %H:%M:%S")

(datetime.datetime(2022, 11, 1, 8, 1, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
 datetime.datetime(2022, 11, 9, 9, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
 11610)

In [2]:
engine = BacktestingEngine()
engine.set_parameters(
    vt_symbols=["BTCUSDT_FUTURE.LOCAL", "BTCUSDT_SPOT.LOCAL"],
    interval=Interval.MINUTE,
    start=datetime(2022, 11, 1),
    end=datetime(2022, 11, 9),
    rates={
        "BTCUSDT_FUTURE.LOCAL": 0/10000,
        "BTCUSDT_SPOT.LOCAL": 0/10000
    },
    slippages={
        "BTCUSDT_FUTURE.LOCAL": 0,
        "BTCUSDT_SPOT.LOCAL": 0
    },
    sizes={
        "BTCUSDT_FUTURE.LOCAL": 1,
        "BTCUSDT_SPOT.LOCAL": 1
    },
    priceticks={
        "BTCUSDT_FUTURE.LOCAL": 0.01,
        "BTCUSDT_SPOT.LOCAL": 0.01
    },
    capital=1_000_000,
)
setting = {
}

engine.add_strategy(MasonStrategy, setting)

In [3]:
#%%
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
engine.show_chart()

2022-11-09 12:24:48.007113	开始加载历史数据
2022-11-09 12:24:49.239144	BTCUSDT_FUTURE.LOCAL加载进度：########## [100%]
2022-11-09 12:24:49.239144	BTCUSDT_FUTURE.LOCAL历史数据加载完成，数据量：11040
2022-11-09 12:24:50.000033	BTCUSDT_SPOT.LOCAL加载进度：########## [100%]
2022-11-09 12:24:50.000033	BTCUSDT_SPOT.LOCAL历史数据加载完成，数据量：11040
2022-11-09 12:24:50.000033	所有历史数据加载完成
2022-11-09 12:24:50.009009	策略初始化完成
2022-11-09 12:24:50.009009	开始回放历史数据
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20378.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20374.0
20369.4
20369.4
20367.5
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20353.2
20342.7
20320.6
20312.4
20312.4
20312.4
20312.4
20312.4
20312.4
20312.4
20318.9
20318.9
20318.9
20318.9
20318.9
20318.9
20318.9
20325.2
20325.2
20325.2
20325.2
203

In [9]:
pd.DataFrame(engine.get_all_trades())

,gateway_name,symbol,exchange,orderid,tradeid,direction,offset,price,volume,datetime
0,BACKTESTING,BTCUSDT_FUTURE,Exchange.LOCAL,1,1,Direction.SHORT,Offset.OPEN,20324.0,1,2022-11-03 12:25:00+08:00


In [24]:
data1[10050:10100]

,open,high,low,close,volume
index,,,,,
2022-11-08 07:31:00,20551.80,20555.00,20548.00,20550.80,225.941
2022-11-08 07:32:00,20550.70,20550.70,20526.10,20538.80,323.078
2022-11-08 07:33:00,20538.80,20547.40,20535.30,20537.80,261.210
2022-11-08 07:34:00,20537.80,20550.10,20537.80,20541.00,248.019
2022-11-08 07:35:00,20540.90,20543.00,20537.90,20540.20,144.200
2022-11-08 07:36:00,20540.10,20545.70,20539.00,20545.70,125.647
2022-11-08 07:37:00,20545.70,20559.90,20543.00,20556.10,435.519
2022-11-08 07:38:00,20556.30,20559.30,20545.60,20545.60,219.166
2022-11-08 07:39:00,20545.60,20557.70,20545.60,20557.70,110.375


In [23]:
20324*(1+0.01*6)

21543.440000000002

In [4]:
df

,trade_count,turnover,commission,slippage,trading_pnl,holding_pnl,total_pnl,net_pnl,balance,return,highlevel,drawdown,ddpercent
date,,,,,,,,,,,,,
2022-11-02,0,0.0,0.0,0,0.0,0.0,0.0,0.0,1000000.0,0.000000,1000000.0,0.0,0.000000
2022-11-03,34,688355.3,0.0,0,-116.1,0.0,-116.1,-116.1,999883.9,-0.000116,1000000.0,-116.1,-0.011610
2022-11-04,60,1217193.6,0.0,0,353.4,0.0,353.4,353.4,1000237.3,0.000353,1000237.3,0.0,0.000000
2022-11-05,126,2665799.8,0.0,0,4.8,0.0,4.8,4.8,1000242.1,0.000005,1000242.1,0.0,0.000000
2022-11-06,58,1233003.6,0.0,0,-56.4,0.0,-56.4,-56.4,1000185.7,-0.000056,1000242.1,-56.4,-0.005639
2022-11-07,34,720697.0,0.0,0,-18.0,0.0,-18.0,-18.0,1000167.7,-0.000018,1000242.1,-74.4,-0.007438
2022-11-08,32,661336.8,0.0,0,60.0,0.0,60.0,60.0,1000227.7,0.000060,1000242.1,-14.4,-0.001440
2022-11-09,0,0.0,0.0,0,0.0,0.0,0.0,0.0,1000227.7,0.000000,1000242.1,-14.4,-0.001440
